In [6]:
import logging
from linkedin_jobs_scraper import LinkedinScraper
from linkedin_jobs_scraper.events import Events, EventData, EventMetrics
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, \
    OnSiteOrRemoteFilters, SalaryBaseFilters

# Change root logger level (default is WARN)
logging.basicConfig(level=logging.INFO)


# Fired once for each successfully processed job
def on_data(data: EventData):
    print('[ON_DATA]', data.title, data.company, data.company_link, data.date, data.link, data.insights ,data.description
          ,len(data.description))


# Fired once for each page (25 jobs)
def on_metrics(metrics: EventMetrics):
    print('[ON_METRICS]', str(metrics))


def on_error(error):
    print('[ON_ERROR]', error)


def on_end():
    print('[ON_END]')


scraper = LinkedinScraper(
    chrome_executable_path=None,  # Custom Chrome executable path (e.g. /foo/bar/bin/chromedriver)
    chrome_binary_location=None,  # Custom path to Chrome/Chromium binary (e.g. /foo/bar/chrome-mac/Chromium.app/Contents/MacOS/Chromium)
    chrome_options=None,  # Custom Chrome options here
    headless=True,  # Overrides headless mode only if chrome_options is None
    max_workers=1,  # How many threads will be spawned to run queries concurrently (one Chrome driver for each thread)
    slow_mo=0.5,  # Slow down the scraper to avoid 'Too many requests 429' errors (in seconds)
    page_load_timeout=40  # Page load timeout (in seconds)    
)

# Add event listeners
scraper.on(Events.DATA, on_data)
scraper.on(Events.ERROR, on_error)
scraper.on(Events.END, on_end)

queries = [
    Query(
        query='Data Scientist at Meta',
        options=QueryOptions(
            locations=['United States'],
            apply_link=False,  # Try to extract apply link (easy applies are skipped). If set to True, scraping is slower because an additional page must be navigated. Default to False.
            skip_promoted_jobs=True,  # Skip promoted jobs. Default to False.
            page_offset=0,  # How many pages to skip
            limit=5,
            filters=QueryFilters(
                company_jobs_url='https://www.linkedin.com/jobs/search'+
                '/?f_C=10667%2C2289109%2C23769%2C27104390%2C2763277%2C76987811%2C289891%2C27046884%2C16159097&geoId=92000000&origin='
                +'COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3988530422%2C3988530419%2C3996899368%2C4005364382%2'
                +'C3988531331%2C3970890472%2C3991150716%2C4002388216%2C4005365153&lipi=urn%3Ali%3Apage%3Ad_flagship3_company%3BnTmFqwERSIG4h%2BDZ72DAjQ%3D%3D',               
                relevance=RelevanceFilters.RECENT,
                time=TimeFilters.MONTH,
                type=[TypeFilters.FULL_TIME],
                on_site_or_remote=None,
                experience=None,
                base_salary=SalaryBaseFilters.SALARY_200K
            )
        )
    )
]

scraper.run(queries)

INFO:li:scraper:('Using strategy AnonymousStrategy',)
INFO:li:scraper:('Starting new query', "Query(query=Data Scientist at Meta options=QueryOptions(limit=5 locations=['United States'] filters=QueryFilters(company_jobs_url=https://www.linkedin.com/jobs/search/?f_C=10667%2C2289109%2C23769%2C27104390%2C2763277%2C76987811%2C289891%2C27046884%2C16159097&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3988530422%2C3988530419%2C3996899368%2C4005364382%2C3988531331%2C3970890472%2C3991150716%2C4002388216%2C4005365153&lipi=urn%3Ali%3Apage%3Ad_flagship3_company%3BnTmFqwERSIG4h%2BDZ72DAjQ%3D%3D relevance=RelevanceFilters.RECENT time=TimeFilters.MONTH base_salary=SalaryBaseFilters.SALARY_200K type=[<TypeFilters.FULL_TIME: 'F'>]) apply_link=False skip_promoted_jobs=True page_offset=0))")
INFO:li:scraper:('Chrome debugger url', 'http://localhost:51379')
INFO:li:scraper:('Websocket debugger url: ', 'ws://localhost:51379/devtools/page/1C9A96A67979F92DDCDE431E9D1F3

[ON_DATA] Data Scientist, Small Business Group Meta  2024-08-05 https://www.linkedin.com/jobs/view/data-scientist-small-business-group-at-meta-3953550500?position=1&pageNum=0&refId=4muh8W3otFmK3%2BA12zZ9ow%3D%3D&trackingId=Y%2Fhj9aw%2B5G%2FH4S82NGKlHA%3D%3D&trk=public_jobs_jserp-result_search-card [] The Small Business Group (SBG) contributes directly to Facebook’s mission by connecting small businesses with their customers and helping them grow through solutions like pages, advertising, and offers. We succeed when we help our customers grow their business. The SBG team is searching for a Data Scientist. The Data Scientist will use data analysis to understand customer profiles, produce reports to track our business, build models to provide insight into the Small Business customer base, identify opportunities, and impact the strategy of our Product, Marketing, and Sales teams. Ideal candidates love breaking down problems, building solutions, delivering actionable and data-driven insight

INFO:li:scraper:('[Data Scientist at Meta][United States][2]', 'Processed')


[ON_DATA] Data Scientist, Product Analytics Meta  2024-08-03 https://www.linkedin.com/jobs/view/data-scientist-product-analytics-at-meta-3876164107?position=2&pageNum=0&refId=4muh8W3otFmK3%2BA12zZ9ow%3D%3D&trackingId=y7k2IdAHUDUSCxQqPWm4nw%3D%3D&trk=public_jobs_jserp-result_search-card [] As a Data Scientist at Meta, you will shape the future of people-facing and business-facing products we build across our entire family of applications (Facebook, Instagram, Messenger, WhatsApp, Oculus). By applying your technical skills, analytical mindset, and product intuition to one of the richest data sets in the world, you will help define the experiences we build for billions of people and hundreds of millions of businesses around the world. You will collaborate on a wide array of product and business problems with a diverse set of cross-functional partners across Product, Engineering, Research, Data Engineering, Marketing, Sales, Finance and others. You will use data and analysis to identify an

INFO:li:scraper:('[Data Scientist at Meta][United States][3]', 'Processed')


[ON_DATA] Data Scientist, Small Business Group Meta  2024-08-04 https://www.linkedin.com/jobs/view/data-scientist-small-business-group-at-meta-3953549601?position=3&pageNum=0&refId=4muh8W3otFmK3%2BA12zZ9ow%3D%3D&trackingId=bV3SumSomy6ien5xbP0T3A%3D%3D&trk=public_jobs_jserp-result_search-card [] The Small Business Group (SBG) contributes directly to Facebook’s mission by connecting small businesses with their customers and helping them grow through solutions like pages, advertising, and offers. We succeed when we help our customers grow their business. The SBG team is searching for a Data Scientist. The Data Scientist will use data analysis to understand customer profiles, produce reports to track our business, build models to provide insight into the Small Business customer base, identify opportunities, and impact the strategy of our Product, Marketing, and Sales teams. Ideal candidates love breaking down problems, building solutions, delivering actionable and data-driven insights, and

ERROR:li:scraper:('[Data Scientist at Meta][United States][4]', 'Timeout on loading job details')
NoneType: None
ERROR:li:scraper:('[Data Scientist at Meta][United States][4]', 'Timeout on loading job details')
NoneType: None
INFO:li:scraper:('[Data Scientist at Meta][United States][4]', 'Processed')


[ON_DATA] Data Scientist, GBG Data Science Meta  2024-08-02 https://www.linkedin.com/jobs/view/data-scientist-gbg-data-science-at-meta-3953545868?position=6&pageNum=0&refId=4muh8W3otFmK3%2BA12zZ9ow%3D%3D&trackingId=Vxvuoi80cwC1go67d5SeBg%3D%3D&trk=public_jobs_jserp-result_search-card [] The massive scale and heavy engagement of the people using our products make the Meta ads platform the most powerful growth vehicle for businesses in the world. The team is driving advancements in ad auction and ad delivery research, developing/vetting advanced advertising tactics, and bootstrapping new products that meet large but nuanced use cases of complex advertisers. In this role, you will delve into the multifaceted ad ranking product stack, employing advanced modeling, and machine learning to unravel the complexities of modern advertising. You must also be able to work with external marketers as this role will give you the opportunity to meet with the most sophisticated advertisers to understand

INFO:li:scraper:('[Data Scientist at Meta][United States][5]', 'Processed')


[ON_DATA] Data Scientist, Product Analytics Meta  2024-08-03 https://www.linkedin.com/jobs/view/data-scientist-product-analytics-at-meta-3811304739?position=7&pageNum=0&refId=4muh8W3otFmK3%2BA12zZ9ow%3D%3D&trackingId=s5lJAHXCJFJy0oEaC%2FdJmw%3D%3D&trk=public_jobs_jserp-result_search-card [] As a Data Scientist at Meta, you will shape the future of people-facing and business-facing products we build across our entire family of applications (Facebook, Instagram, Messenger, WhatsApp, Oculus). By applying your technical skills, analytical mindset, and product intuition to one of the richest data sets in the world, you will help define the experiences we build for billions of people and hundreds of millions of businesses around the world. You will collaborate on a wide array of product and business problems with a diverse set of cross-functional partners across Product, Engineering, Research, Data Engineering, Marketing, Sales, Finance and others. You will use data and analysis to identify 